![image](https://i.ibb.co/d03v41b/architecture-diagramxx.png)

In [34]:
import boto3
import json
import time
import random
from botocore.exceptions import ClientError

region_aws = 'us-east-1'

session = boto3.Session(profile_name='default')
ec2 = session.resource('ec2', region_name=region_aws)
ec2_client = session.client('ec2', region_name=region_aws)
elbv2 = session.client('elbv2', region_name=region_aws)
s3 = session.resource('s3', region_name=region_aws)

In [24]:
# Create bucket
bucket = 'cookbook-bucket-{}'.format(random.randint(10000, 1000000))
bucket = s3.create_bucket(Bucket=bucket)
print(f"Bucket: {bucket}")

Bucket: s3.Bucket(name='cookbook-bucket-329036')


In [2]:
# Create VPC
vpc_applic = ec2.create_vpc(CidrBlock='10.10.0.0/16')
vpc_applic.create_tags(Tags=[{"Key": "Name", "Value": "Applicance"}]);

In [3]:
# Create 2 subnets in VPC Applicance
sn_appliance_transit_a = ec2.create_subnet(
    VpcId=vpc_applic.id,
    CidrBlock='10.10.1.0/24',
    AvailabilityZone=region_aws+'a'
)
sn_appliance_transit_b = ec2.create_subnet(
    VpcId=vpc_applic.id,
    CidrBlock='10.10.2.0/24',
    AvailabilityZone=region_aws+'b'
)

In [4]:
# Tag subnets
sn_appliance_transit_a.create_tags(Tags=[{"Key": "Name", "Value": "Public-Subnet"}])
sn_appliance_transit_b.create_tags(Tags=[{"Key": "Name", "Value": "Private-Subnet"}])

[ec2.Tag(resource_id='subnet-0740ce2a7e9681695', key='Name', value='Private-Subnet')]

In [5]:
# Create route table for the VPC Applicance
rt_appliance_transit_a = vpc_applic.create_route_table()
rt_appliance_transit_b = vpc_applic.create_route_table()

# Tags Route Tables
rt_appliance_transit_a.create_tags(Tags=[{"Key": "Name", "Value": "Public-RT"}])
rt_appliance_transit_b.create_tags(Tags=[{"Key": "Name", "Value": "Private-RT"}])

[ec2.Tag(resource_id='rtb-0d095a64a1fde27e8', key='Name', value='Private-RT')]

In [6]:
# Attach the route table to the subnets
rt_appliance_transit_a.associate_with_subnet(SubnetId=sn_appliance_transit_a.id)
rt_appliance_transit_b.associate_with_subnet(SubnetId=sn_appliance_transit_b.id)

ec2.RouteTableAssociation(id='rtbassoc-0ba2fcff789034a86')

In [7]:
# Create Internet Gateway
igw = ec2.create_internet_gateway()

# Attach Internet Gateway to VPC Applicance
vpc_applic.attach_internet_gateway(InternetGatewayId=igw.id);

In [8]:
# Crear una Elastic IP address
elastic_ip_1 = ec2_client.allocate_address(Domain='vpc')

In [9]:
# Crear un NAT gateway en VPC Applicance
natgateway_1 = ec2_client.create_nat_gateway(
    AllocationId=elastic_ip_1['AllocationId'],
    SubnetId=sn_appliance_transit_a.id
)

In [10]:
# Añada una ruta a la tabla de enrutamiento de la VPC Applicance para el internet natgateway
rt_appliance_transit_a.create_route(
    DestinationCidrBlock='0.0.0.0/0',
    GatewayId=igw.id
)

ec2.Route(route_table_id='rtb-0095dd7d7585af23d', destination_cidr_block='0.0.0.0/0')

In [11]:
# Añada una ruta a la tabla de enrutamiento de la VPC Applicance para el NAT gateway
rt_appliance_transit_b.create_route(
    DestinationCidrBlock='0.0.0.0/0',
    NatGatewayId=natgateway_1['NatGateway']['NatGatewayId']
)

ec2.Route(route_table_id='rtb-0d095a64a1fde27e8', destination_cidr_block='0.0.0.0/0')

In [39]:
# Create a gateway endpoint in your VPC and associate the endpoint with the isolated route tables
response = ec2_client.create_vpc_endpoint(
    VpcId=vpc_applic.id,
    ServiceName='com.amazonaws.' + region_aws + '.s3',
    RouteTableIds=[
        rt_appliance_transit_a.id
    ]       
)
vpc_endpoint_id = response['VpcEndpoint']['VpcEndpointId']

In [40]:
policy = {
    "Statement": [
        {
            "Action": "s3:*",
            "Effect": "Deny",
            "Resource": f"arn:aws:s3:::{bucket.name}/*",
            "Condition": {
                "StringNotEquals": {
                    "aws:sourceVpce": vpc_endpoint_id
                }
            },
            "Principal": "*"
        }
    ]
}

bucket.Policy().put(Policy=json.dumps(policy))

{'ResponseMetadata': {'RequestId': '5E9R41K33W969P77',
  'HostId': 'POYaC7PDJRLSXvjX6+Cjuay6FJRtDMXWBS21OML0Cuc4uMjjJzhxCD/Xnnlbjt1ceblrP42Bg9I=',
  'HTTPStatusCode': 204,
  'HTTPHeaders': {'x-amz-id-2': 'POYaC7PDJRLSXvjX6+Cjuay6FJRtDMXWBS21OML0Cuc4uMjjJzhxCD/Xnnlbjt1ceblrP42Bg9I=',
   'x-amz-request-id': '5E9R41K33W969P77',
   'date': 'Mon, 31 Jul 2023 15:40:18 GMT',
   'server': 'AmazonS3'},
  'RetryAttempts': 0}}

In [42]:
try:
    key_name = 'ip-ranges.json'
    response = bucket.upload_file(key_name, key_name)
    pprint(response)
except ClientError as e:
    print(f"Error: {e}")
except Exception as e:
    print(f"Error: {e}")

Error: Failed to upload ip-ranges.json to cookbook-bucket-329036/ip-ranges.json: An error occurred (AccessDenied) when calling the PutObject operation: Access Denied


In [50]:
print("En la terminal ejecute el siguiente comando:")
print(f"    python3 create_ec2_ssm.py --vpc {vpc_applic.id} --subnet {sn_appliance_transit_a.id} --region {region_aws}")

En la terminal ejecute el siguiente comando:
    python3 create_ec2_ssm.py --vpc vpc-08bdc04dfe3308d3b --subnet subnet-0427c3d261a2cc764 --region us-east-1


In [51]:
bucket.name

'cookbook-bucket-329036'